# opensearch loading

The purpose of this notebook is to load an initial set of data into opensearch so we can validate the initial set of queries.
We'll take a look at data for 2024 in the process.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
data_root = "../../data/longeval"
! tree {data_root} -L 3

../../data/longeval
├── LongEval 2024 Test Collection Readme.pdf
├── LongEval 2024 Train Collection Readme.pdf
├── LongEval Test Collection
│   ├── 2023_06
│   │   ├── English
│   │   ├── French
│   │   └── count_docs.sh
│   └── 2023_08
│       ├── English
│       ├── French
│       └── count_docs.sh
├── LongEval Train Collection
│   └── 2023_01
│       ├── English
│       ├── French
│       └── count_docs.sh
├── id_urls_2023_01.txt
├── id_urls_2023_06.txt
├── id_urls_2023_08.txt
└── opensearch-v1
    ├── batch_metrics_enabled.conf
    ├── logging_enabled.conf
    ├── nodes
    │   └── 0
    ├── performance_analyzer_enabled.conf
    ├── rca_enabled.conf
    └── thread_contention_monitoring_enabled.conf

14 directories, 13 files


In [3]:
! cat {data_root}/id_urls_2023_01.txt | head -n 10

doc012310000001	https://nationalinterest.org/blog/reboot/forget-nukes-tunnels-are-north-koreas-secret-weapon-164295
doc012310000002	https://ppmforums.com/viewtopic.php?t=46501&view=previous
doc012310000003	https://www.gettyimages.nl/detail/nieuwsfoto%27s/lawyer-james-b-donovan-widely-known-for-negotiating-the-nieuwsfotos/540536060
doc012310000004	https://nationalinterest.org/feature/what-rex-tillersons-nomination-means-russia-policy-18726
doc012310000005	https://en.pokechange.net/@Issan/collection
doc012310000006	http://www.llu.edu/pages/faculty/directory/portfolio_activity.php?catid=5&eid=1a30481&uid=junternaehrer
doc012310000007	https://wikimili.com/en/Kurt_Meyer
doc012310000008	http://www.historycommons.org/context.jsp?item=moorheadcensor72
doc012310000009	https://farsight.org/demo/Time_Cross_Project/2018/Time_Cross_January_2018_Events.html
doc012310000010	https://jnccn.org/abstract/journals/jnccn/15/8/article-p1028.xml?result=2&rskey=26Qf0J
cat: write error: Broken pipe


In [4]:
from longeval.utils import get_spark

data_root = "../../data/longeval"

spark = get_spark()
spark

24/12/19 00:02:21 WARN Utils: Your hostname, daphne-major resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/19 00:02:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/anthony/omscs/dsgt-clef/venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/anthony/.ivy2/cache
The jars for the packages stored in: /home/anthony/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-52338051-d53c-4d86-a20c-9a707a001d85;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.18.0 in central
	found commons-io#commons-io;2.11.0 in central
	found org.glassfish.jaxb#txw2;3.0.2 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.3.0 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 in central
downloading https://repo1.maven.org/maven2/com/databricks/spark-xml_2.12/0.18.0/spark-xml_2.12-0.18.0.jar ...
	[SUCCESSFUL ] com.databricks#spark-xml_2.12;0.18.0!spark-xml_2.12.jar (95ms)
:: resolution report :: resolve 1123ms :: artifacts dl 99ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.18.0 from central in [default]
	commons-io#commons-io;2.11.0 from central in [default]
	org.apache.ws.xmlschema#xmlsche

In [5]:
! cat {data_root}/"LongEval Test Collection"/2023_06/English/Documents/Trec/collector_kodicare_4.txt | head -n 10

<DOC>
<DOCNO>doc062300400001</DOCNO>
<DOCID>doc062300400001</DOCID>
<TEXT>
Request for Quotation LLD
Renault Traffic
- Request for a long-term rental quotation
Renault Traffic //
Your quote request LLD / LOA
Renault Renault TRAFFIC
cat: write error: Broken pipe


In [6]:
# find all the unique tags in the dataset
! cat {data_root}/"LongEval Test Collection"/2023_06/English/Documents/Trec/collector_kodicare_3.txt | \
    grep -oP "<[^>\/]+>" | sort | uniq -c | sort -nr | head -n 8

  17887 <TEXT>
  17887 <DOCNO>
  17887 <DOCID>
  17887 <DOC>
     82 <br>
     32 <p>
     27 <strong>
     26 <reference>


In [159]:
! cat {data_root}/"LongEval Test Collection"/2023_06/English/Documents/Trec/* | \
    grep -oaE "<[^>\/]+>" | sort | uniq -c | sort -nr | head -n 8

1773519 <TEXT>
1773519 <DOCNO>
1773519 <DOCID>
1773519 <DOC>
   5557 <p>
   5244 <br>
   2790 <strong>
   2590 < >
sort: write failed: 'standard output': Broken pipe
sort: write error


In [145]:
from longeval.collection import Collection

# according to a simple grep, there are 1759690 documents in the collection
train_eng = Collection(spark, f"{data_root}/LongEval Test Collection/2023_06/English")
docs = train_eng.documents.cache()
docs.count()

191827

In [148]:
docs.show(5, vertical=True, truncate=80)

-RECORD 0-------------------------------------------------------------------------------------------
 DOCID           | doc062300100001                                                                  
 DOCNO           | doc062300100001                                                                  
 TEXT            | \n- siratunnabi.mp3 http://www.4shared.com/audio/kgUtPw_O/MESTICA_-__Sir\n       
 _corrupt_record | NULL                                                                             
-RECORD 1-------------------------------------------------------------------------------------------
 DOCID           | doc062300100002                                                                  
 DOCNO           | doc062300100002                                                                  
 TEXT            | \nSurpervivalism\n|\nNew Land\n"\nWhen I\nwas in the army, it was March or Cr... 
 _corrupt_record | NULL                                                                    

In [149]:
import re

sub = docs.where("NOT (length(DOCID) > 0 AND length(DOCNO) > 0 and length(TEXT) > 0)")
print(sub.count())
for row in sub.limit(5).collect():
    print(row)
    print(
        [
            x[:50]
            for x in re.findall("<TEXT>(.*?)</TEXT>", row._corrupt_record, re.DOTALL)
        ]
    )
    print("-" * 80)

0


In [157]:
docs.groupBy("DOCNO").count().count()

191827